In [9]:
import tushare as ts
import numpy as mp
import pandas as pd
import os
import jqdatasdk     # JoinQuant SDK
import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [18]:
# 登录认证
jqdatasdk.auth("18867905508", "Ljy19950217") # user & password
count_num = jqdatasdk.get_query_count() # JQ总共能查多少data 还剩能查多少data
print(count_num)

{'total': 500000, 'spare': 499605}


In [19]:
# 获取数据
# 研报中说明：将 2013-01-01 至 2017-12-31 作为训练集，样本数据频率为周，共计 256 周。
data = jqdatasdk.get_price("000300.XSHG", start_date="2013-01-01", end_date="2017-12-31", frequency="7d")
data.head()

/Users/lijunyu/opt/anaconda3/envs/LI38/lib/python3.8/site-packages/jqdatasdk/api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


,open,close,high,low,volume,money
2013-01-14,2551.81,2577.72,2583.63,2473.70,5.541036e+10,5.602669e+11
2013-01-23,2579.13,2607.46,2620.03,2539.67,6.056519e+10,6.064508e+11
2013-02-01,2606.43,2743.32,2744.32,2564.21,6.625742e+10,6.831217e+11
2013-02-19,2753.69,2685.61,2791.30,2674.34,6.513217e+10,6.801547e+11
2013-02-28,2686.09,2673.33,2703.10,2564.67,5.141966e+10,5.389911e+11


In [26]:
# 获取的数据存为本地csv文件 以便读取修改dataframe
# 获取train数据
data.to_csv("/Users/lijunyu/Desktop/Python/Assignment_3/train2013.csv")

In [31]:
# 获取测试集数据
# 策略运行以来，测试集(2018-01-02 至 2021-05-10)
data_test = jqdatasdk.get_price("000300.XSHG", start_date="2018-01-02", end_date="2021-05-10", frequency="7d")
data_test.head()

/Users/lijunyu/opt/anaconda3/envs/LI38/lib/python3.8/site-packages/jqdatasdk/api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


,open,close,high,low,volume,money
2018-01-10,4045.21,4207.81,4211.05,4045.21,1.046375e+11,1.347088e+12
2018-01-19,4197.11,4285.40,4316.57,4181.96,1.161171e+11,1.559670e+12
2018-01-30,4276.48,4256.10,4403.34,4251.57,1.299067e+11,1.692011e+12
2018-02-08,4234.11,4012.05,4287.86,3974.68,1.219936e+11,1.582146e+12
2018-02-26,3896.17,4118.42,4128.74,3759.15,8.493310e+10,1.079766e+12


In [32]:
data_test.to_csv("/Users/lijunyu/Desktop/Python/Assignment_3/test2018.csv") 

In [29]:
# Train data
# Clean data and re-format
train_set = pd.read_csv(os.getcwd()+'/train2013.csv')
train_set = train_set[['datetime', 'open', 'high', 'low', 'close', 'volume', 'money']]
train300 = train_set
train300.to_csv("/Users/lijunyu/Desktop/Python/Assignment_3/train.csv")
train300        # train data for weekly_300

,datetime,open,high,low,close,volume,money
0,2013-01-14,2551.81,2583.63,2473.70,2577.72,5.541036e+10,5.602669e+11
1,2013-01-23,2579.13,2620.03,2539.67,2607.46,6.056519e+10,6.064508e+11
2,2013-02-01,2606.43,2744.32,2564.21,2743.32,6.625742e+10,6.831217e+11
3,2013-02-19,2753.69,2791.30,2674.34,2685.61,6.513217e+10,6.801547e+11
4,2013-02-28,2686.09,2703.10,2564.67,2673.33,5.141966e+10,5.389911e+11
...,...,...,...,...,...,...,...
169,2017-11-24,4063.49,4260.64,4057.49,4104.20,1.126449e+11,1.545622e+12
170,2017-12-05,4088.93,4088.93,3982.32,4040.17,8.321716e+10,1.100689e+12
171,2017-12-14,4027.12,4070.15,3955.44,4026.15,6.286258e+10,8.902225e+11
172,2017-12-25,4018.73,4086.93,3967.93,4041.54,5.972680e+10,8.451944e+11


In [30]:
# Test data
# Clean data and re-format
test_set = pd.read_csv(os.getcwd()+'/test2018.csv')
test_set = test_set[['datetime', 'open', 'high', 'low', 'close', 'volume', 'money']]
test_set_300 = test_set
test_set_300.to_csv("/Users/lijunyu/Desktop/Python/Assignment_3/test.csv")
test_set_300              # train data for weekly_300

,datetime,open,high,low,close,volume,money
0,2013-01-14,2551.81,2583.63,2473.70,2577.72,5.541036e+10,5.602669e+11
1,2013-01-23,2579.13,2620.03,2539.67,2607.46,6.056519e+10,6.064508e+11
2,2013-02-01,2606.43,2744.32,2564.21,2743.32,6.625742e+10,6.831217e+11
3,2013-02-19,2753.69,2791.30,2674.34,2685.61,6.513217e+10,6.801547e+11
4,2013-02-28,2686.09,2703.10,2564.67,2673.33,5.141966e+10,5.389911e+11
...,...,...,...,...,...,...,...
169,2017-11-24,4063.49,4260.64,4057.49,4104.20,1.126449e+11,1.545622e+12
170,2017-12-05,4088.93,4088.93,3982.32,4040.17,8.321716e+10,1.100689e+12
171,2017-12-14,4027.12,4070.15,3955.44,4026.15,6.286258e+10,8.902225e+11
172,2017-12-25,4018.73,4086.93,3967.93,4041.54,5.972680e+10,8.451944e+11
